In [ ]:
import numpy as np
import pandas as pd
from dataclasses import asdict

from use_max import max_value_of_function

## Finding the maximum of a quadratic using Quantum max-finding

We demonstrate using the qubrabench `max` function by using it to find the maximum of the following quadratic
$$f(x) = 4 x - x^2$$

In [ ]:
def quadratic(x: float):
    return 4 * x - x**2

In [ ]:
%psource max_value_of_function

In [ ]:
# Find `x` that maximizes the above quadratic, in range [-10, 10]
x_best, stats = max_value_of_function(np.linspace(-10, 10, num=1000), quadratic)
x_best

We can aggregate the costs of the quantum algorithm by combining the classical and quantum query counts with an appropriate "scaling factor". In this example, we assume that each quantum query is _twice_ as expensive as a classical one.

In [ ]:
cq = 2  # multiplier for quantum queries
classical_cost = stats.classical_actual_queries
quantum_cost = (
    stats.quantum_expected_classical_queries
    + cq * stats.quantum_expected_quantum_queries
)

In [ ]:
classical_cost, quantum_cost

## Benchmarking
We can now benchmark the max-finding for increasingly large search spaces. We vary the number of samples from $2^4$ till $2^{18}$, in the range $[-10, 10]$.

We collect all this data into a pandas DataFrame.

In [ ]:
data = []
for pwr in range(4, 19):
    for _ in range(2): # repeat twice
        # search over 2**pwr uniformly distributed values in [-10, 10]
        n_samples = 2**pwr
        x_best, stats = max_value_of_function(np.linspace(-10, 10, num=n_samples), quadratic)
    
        # process stats
        stats = asdict(stats)
        stats["n"] = n_samples
        data.append(stats)
data = pd.DataFrame([list(row.values()) for row in data], columns=list(data[0].keys()))
data

In [ ]:
from qubrabench.utils.plotting_strategy import PlottingStrategy

In [ ]:
%pdoc PlottingStrategy

In [ ]:
class Plotter(PlottingStrategy):
    def __init__(self):
        self.colors[""] = "blue"

    def get_plot_group_column_names(self):
        return []

    def get_data_group_column_names(self):
        return []

    def compute_aggregates(self, data, *, quantum_factor):
        # compute combined query costs of quantum search
        c = data["quantum_expected_classical_queries"]
        q = data["quantum_expected_quantum_queries"]
        data["classical_cost"] = data["classical_actual_queries"]
        data["quantum_cost"] = c + quantum_factor * q
        return data

    def x_axis_column(self):
        return "n"

    def x_axis_label(self):
        return "$n$"

    def y_axis_label(self):
        return "Queries"

    def get_column_names_to_plot(self):
        return {
            "classical_cost": ("Classical Queries", "o"),
            "quantum_cost": ("Quantum Queries", "x"),
        }

In [ ]:
Plotter().plot(data, quantum_factor=2)